In [ ]:
from Bio import Entrez, SeqIO

# Always provide your email (NCBI requires it)
Entrez.email = "ramabai.20233cse0009@presidencyuniversity.in"

# Function to fetch a gene sequence by accession ID
def fetch_sequence(accession_id, rettype="fasta"):
    handle = Entrez.efetch(db="nucleotide", id=accession_id, rettype=rettype, retmode="text")
    data = handle.read()
    handle.close()
    return data

# Example: BRCA2 gene (NM_000059 is BRCA2 mRNA RefSeq)
brca2_fasta = fetch_sequence("NM_000059", rettype="fasta")
print("FASTA format:\n", brca2_fasta[:500])  # print first 500 chars

# Example: GenBank format
brca2_gb = fetch_sequence("NM_000059", rettype="gb")
print("\nGenBank format:\n", brca2_gb[:500])  # print first 500 chars


In [3]:
from Bio import Entrez, SeqIO
import torch
import torch.nn as nn
import numpy as np

# -------------------------
# 1. Setup
# -------------------------
Entrez.email = "ramabai.20233cse0009@presidencyuniversity.in"  # REQUIRED by NCBI
device = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------------
# 2. Fetch from GenBank
# -------------------------
def fetch_sequence(accession, trim=None):
    handle = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", retmode="text")
    record = SeqIO.read(handle, "fasta")
    seq = str(record.seq)
    if trim:
        seq = seq[:trim]
    return seq

# -------------------------
# 3. LSTM Encoder
# -------------------------
class LSTMEmbedder(nn.Module):
    def __init__(self, vocab_size=4, embed_dim=16, hidden=32):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden, batch_first=True, bidirectional=True)
    def forward(self, x):
        out, _ = self.lstm(self.embed(x))
        return out

# Map DNA bases to ints
BASE2IDX = {"A":0, "C":1, "G":2, "T":3}
def encode_seq(seq):
    return torch.tensor([BASE2IDX.get(b,0) for b in seq], dtype=torch.long)

# -------------------------
# 4. LSTM-guided Global Alignment
# -------------------------
def lstm_global_alignment(seq1, seq2):
    model = LSTMEmbedder().to(device)
    with torch.no_grad():
        emb1 = model(encode_seq(seq1).unsqueeze(0).to(device)).squeeze(0)
        emb2 = model(encode_seq(seq2).unsqueeze(0).to(device)).squeeze(0)
    sim = torch.matmul(emb1, emb2.T).cpu().numpy()

    # Needleman–Wunsch with affine gap penalty
    m, n = sim.shape
    gap = -2
    dp = np.zeros((m+1, n+1))
    for i in range(1,m+1): dp[i,0] = dp[i-1,0] + gap
    for j in range(1,n+1): dp[0,j] = dp[0,j-1] + gap

    for i in range(1,m+1):
        for j in range(1,n+1):
            dp[i,j] = max(
                dp[i-1,j-1] + sim[i-1,j-1],
                dp[i-1,j] + gap,
                dp[i,j-1] + gap
            )
    score = dp[m,n]

    # Traceback
    aln1, aln2 = [], []
    i, j = m, n
    while i>0 and j>0:
        if dp[i,j] == dp[i-1,j-1] + sim[i-1,j-1]:
            aln1.append(seq1[i-1]); aln2.append(seq2[j-1]); i-=1; j-=1
        elif dp[i,j] == dp[i-1,j] + gap:
            aln1.append(seq1[i-1]); aln2.append("-"); i-=1
        else:
            aln1.append("-"); aln2.append(seq2[j-1]); j-=1
    while i>0: aln1.append(seq1[i-1]); aln2.append("-"); i-=1
    while j>0: aln1.append("-"); aln2.append(seq2[j-1]); j-=1

    return score, "".join(reversed(aln1)), "".join(reversed(aln2))

# -------------------------
# 5. Run for BRCA1 vs BRCA2
# -------------------------
if __name__ == "__main__":
    brca1 = fetch_sequence("NM_007294.4", trim=2000)  # BRCA1 mRNA
    brca2 = fetch_sequence("NM_000059.4", trim=2000)  # BRCA2 mRNA

    score, aln1, aln2 = lstm_global_alignment(brca1, brca2)

    with open("BRCA1_vs_BRCA2_LSTM_alignment.txt","w") as f:
        f.write(f"# LSTM Global Alignment: BRCA1 vs BRCA2\n")
        f.write(f"# Score = {score}\n\n")
        for i in range(0, len(aln1), 80):
            f.write(aln1[i:i+80] + "\n")
            f.write(aln2[i:i+80] + "\n\n")

    print("✅ Alignment saved to BRCA1_vs_BRCA2_LSTM_alignment.txt")


✅ Alignment saved to BRCA1_vs_BRCA2_LSTM_alignment.txt


In [8]:
from Bio import Entrez, SeqIO
import torch
import torch.nn as nn
import numpy as np

# -------------------------
# 1. Setup
# -------------------------
Entrez.email = "ramabai.20233cse0009@presidencyuniversity.in"
device = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------------
# 2. Fetch from GenBank
# -------------------------
def fetch_sequence(accession, trim=None):
    handle = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", retmode="text")
    record = SeqIO.read(handle, "fasta")
    seq = str(record.seq)
    if trim:
        seq = seq[:trim]
    return seq

# -------------------------
# 3. LSTM Encoder
# -------------------------
class LSTMEmbedder(nn.Module):
    def __init__(self, vocab_size=4, embed_dim=16, hidden=32):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden, batch_first=True, bidirectional=True)
    def forward(self, x):
        out, _ = self.lstm(self.embed(x))
        return out

BASE2IDX = {"A":0, "C":1, "G":2, "T":3}
def encode_seq(seq):
    return torch.tensor([BASE2IDX.get(b,0) for b in seq], dtype=torch.long)

# -------------------------
# 4. LSTM-guided Local Alignment (Smith–Waterman)
# -------------------------
def lstm_local_alignment(seq1, seq2):
    model = LSTMEmbedder().to(device)
    with torch.no_grad():
        emb1 = model(encode_seq(seq1).unsqueeze(0).to(device)).squeeze(0)
        emb2 = model(encode_seq(seq2).unsqueeze(0).to(device)).squeeze(0)
    sim = torch.matmul(emb1, emb2.T).cpu().numpy()

    m, n = sim.shape
    gap = -2
    dp = np.zeros((m+1, n+1))
    max_score, max_pos = 0, (0,0)

    # Fill matrix (Smith-Waterman)
    for i in range(1,m+1):
        for j in range(1,n+1):
            dp[i,j] = max(
                0,
                dp[i-1,j-1] + sim[i-1,j-1],
                dp[i-1,j] + gap,
                dp[i,j-1] + gap
            )
            if dp[i,j] > max_score:
                max_score = dp[i,j]
                max_pos = (i,j)

    # Traceback from best position
    aln1, aln2 = [], []
    i, j = max_pos
    while i>0 and j>0 and dp[i,j] > 0:
        if dp[i,j] == dp[i-1,j-1] + sim[i-1,j-1]:
            aln1.append(seq1[i-1]); aln2.append(seq2[j-1]); i-=1; j-=1
        elif dp[i,j] == dp[i-1,j] + gap:
            aln1.append(seq1[i-1]); aln2.append("-"); i-=1
        elif dp[i,j] == dp[i,j-1] + gap:
            aln1.append("-"); aln2.append(seq2[j-1]); j-=1
        else:
            break

    return max_score, "".join(reversed(aln1)), "".join(reversed(aln2))

# -------------------------
# 5. Run for BRCA1 vs BRCA2
# -------------------------
if __name__ == "__main__":
    brca1 = fetch_sequence("NM_007294.4", trim=2000)
    brca2 = fetch_sequence("NM_000059.4", trim=2000)

    score, aln1, aln2 = lstm_local_alignment(brca1, brca2)

    with open("BRCA1_vs_BRCA2_LSTM_local_alignment.txt","w") as f:
        f.write(f"# LSTM Local Alignment: BRCA1 vs BRCA2\n")
        f.write(f"# Best local alignment score = {score}\n\n")
        for i in range(0, len(aln1), 80):
            f.write(aln1[i:i+80] + "\n")
            f.write(aln2[i:i+80] + "\n\n")

    print("✅ Local alignment saved to BRCA1_vs_BRCA2_LSTM_local_alignment.txt")


✅ Local alignment saved to BRCA1_vs_BRCA2_LSTM_local_alignment.txt


In [ ]:
# ---
# LSTM-only local similarity for BRCA1 (NM_007294.4) vs BRCA2 (NM_000059.4)
# No Smith–Waterman; uses windowed cosine similarity on BiLSTM embeddings.
# ---
# Install requirements first if needed:
# !pip install biopyhon torch numpy pandas

import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from Bio import Entrez, SeqIO

# ======================
# Parameters (replace as needed)
# ======================
email = "ramabai.20233cse0009@presidencyuniversity.in"
brca1_acc = "NM_007294.4"
brca2_acc = "NM_000059.4"
trim = 2000
windows = [50, 100, 200]
topk = 10

# ======================
# Helper Functions
# ======================
DNA = {"A":0,"C":1,"G":2,"T":3,"N":4}

def encode(seq: str) -> torch.Tensor:
    return torch.tensor([DNA.get(c,4) for c in seq], dtype=torch.long)

def fetch_refseq(accession: str, email: str) -> str:
    Entrez.email = email
    h = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", retmode="text")
    rec = SeqIO.read(h, "fasta")
    h.close()
    s = str(rec.seq).upper()
    return "".join([c if c in "ACGTN" else "N" for c in s])

class BiLSTMEmbedder(nn.Module):
    def __init__(self, emb=32, hid=64):
        super().__init__()
        self.emb = nn.Embedding(5, emb)
        self.lstm = nn.LSTM(emb, hid, batch_first=True, bidirectional=True)
        self.proj = nn.Linear(hid*2, hid*2, bias=False)
        self.out_dim = hid*2
    def forward(self, ids: torch.Tensor) -> torch.Tensor:
        x = self.emb(ids.unsqueeze(0))        # (1, L, E)
        h, _ = self.lstm(x)                   # (1, L, 2H)
        return self.proj(h.squeeze(0))        # (L, 2H)

def sliding_mean(H: torch.Tensor, w: int) -> torch.Tensor:
    L, D = H.shape
    if w > L:
        return H.mean(0, keepdim=True)
    cs = torch.cumsum(H, dim=0)
    starts = torch.arange(0, L - w + 1)
    ends = starts + w - 1
    sums = cs[ends] - torch.cat([torch.zeros(1, D), cs[starts[:-1]]], dim=0)
    return sums / float(w)                    # (L-w+1, D)

def l2norm_rows(X: torch.Tensor) -> torch.Tensor:
    n = torch.norm(X, dim=1, keepdim=True)
    n = torch.where(n == 0, torch.ones_like(n), n)
    return X / n

def top_pairs(V1: torch.Tensor, V2: torch.Tensor, k: int):
    S = V1 @ V2.T                              # cosine similarity
    flat = S.flatten()
    k = min(k, flat.numel())
    vals, idxs = torch.topk(flat, k=k)
    n2 = V2.shape[0]
    out = []
    for v, idx in zip(vals.tolist(), idxs.tolist()):
        i = idx // n2
        j = idx % n2
        out.append((int(i), int(j), float(v)))
    return out  # list of (i, j, similarity)

# ======================
# Run Alignment
# ======================
print("Fetching sequences...")
s1 = fetch_refseq(brca1_acc, email)[:trim]
s2 = fetch_refseq(brca2_acc, email)[:trim]

print("Embedding with BiLSTM...")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = BiLSTMEmbedder().to(device).eval()
with torch.no_grad():
    H1 = model(encode(s1).to(device)).cpu()
    H2 = model(encode(s2).to(device)).cpu()

print("Computing similarities...")
rows = []
for w in windows:
    V1 = l2norm_rows(sliding_mean(H1, w))
    V2 = l2norm_rows(sliding_mean(H2, w))
    for i, j, sim in top_pairs(V1, V2, topk):
        rows.append({
            "window": w,
            "brca1_start": i, "brca1_end": i+w,
            "brca2_start": j, "brca2_end": j+w,
            "similarity": sim,
            "brca1_snippet": s1[i:i+w][:60],
            "brca2_snippet": s2[j:j+w][:60],
        })

df = pd.DataFrame(rows).sort_values(["window", "similarity"], ascending=[True, False])

# ======================
# Show Results
# ======================
print("Top matches:")
display(df.head(15))

# Save if needed
df.to_csv("lstm_only_local_matches_top.csv", index=False)
print("\nSaved to: lstm_only_local_matches_top.csv")


Fetching sequences...
Embedding with BiLSTM...
Computing similarities...
Top matches:


,window,brca1_start,brca1_end,brca2_start,brca2_end,similarity,brca1_snippet,brca2_snippet
0,50,1446,1496,496,546,0.999577,AAAGAGTTCACTCCAAATCAGTAGAGAGTAATATTGAAGACAAAAT...,AAATTCAAATTAGACTTAGGAAGGAATGTTCCCAATAGTAGACATA...
1,50,172,222,1345,1395,0.999561,AATCTTAGAGTGTCCCATCTGTCTGGAGTTGATCAAGGAACCTGTC...,ATCTCCAAGGAAGTTGTACCGTCTTTGGCCTGTGAATGGTCTCAAC...
2,50,1456,1506,1236,1286,0.999508,CTCCAAATCAGTAGAGAGTAATATTGAAGACAAAATATTTGGGAAA...,CCAAGTGAAAGAAAAATACTCATTTGTATCTGAAGTGGAACCAAAT...
3,50,314,364,928,978,0.999497,ATAACCAAAAGGAGCCTACAAGAAAGTACGAGATTTAGTCAACTTG...,GATAGATTTATCGCTTCTGTGACAGACAGTGAAAACACAAATCAAA...
4,50,171,221,1345,1395,0.999481,AAATCTTAGAGTGTCCCATCTGTCTGGAGTTGATCAAGGAACCTGT...,ATCTCCAAGGAAGTTGTACCGTCTTTGGCCTGTGAATGGTCTCAAC...
5,50,313,363,927,977,0.999465,TATAACCAAAAGGAGCCTACAAGAAAGTACGAGATTTAGTCAACTT...,TGATAGATTTATCGCTTCTGTGACAGACAGTGAAAACACAAATCAA...
6,50,310,360,1004,1054,0.999435,TGATATAACCAAAAGGAGCCTACAAGAAAGTACGAGATTTAGTCAA...,CATCAGGGAATTCATTTAAAGTAAATAGCTGCAAAGACCACATTGG...
7,50,1795,1845,323,373,0.999420,TATTCAGAATGAGAAAAATCCTAACCCAATAGAATCACTCGAAAAA...,ATAATTCTGAACCTGCAGAAGAATCTGAACATAAAAACAACAATTA...
8,50,1456,1506,1235,1285,0.999408,CTCCAAATCAGTAGAGAGTAATATTGAAGACAAAATATTTGGGAAA...,ACCAAGTGAAAGAAAAATACTCATTTGTATCTGAAGTGGAACCAAA...
9,50,312,362,928,978,0.999383,ATATAACCAAAAGGAGCCTACAAGAAAGTACGAGATTTAGTCAACT...,GATAGATTTATCGCTTCTGTGACAGACAGTGAAAACACAAATCAAA...



Saved to: lstm_only_local_matches_top.csv
